In [1]:
import utility as ut
import numpy as np
import matplotlib.pyplot as plt
import random
import copy
import math

In [2]:
# File paths
DIRECTORY = "data/"
GANTRY_CRANE_PARAMETERS_JSON_PATH = "gantry_crane_parameters.json"
RESULTS_PATH = "FPA_results/"

In [3]:
# Load data
datasets = ut.load_excel_data(DIRECTORY)

In [4]:
DURATION = 15  # duration in seconds
DT = 0.0001  # time increment in seconds
# Create a time array
time_array = np.arange(0, DURATION + DT, DT)
NUM_STEPS = len(time_array)

In [5]:
interpolated_datasets = []
for i in range(len(datasets)):
    new_trolley_position = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_position"]
    )
    new_cable_length = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["cable_length"]
    )
    new_sway_angle = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["sway_angle"]
    )
    new_trolley_speed = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_position_first_derivative"]
    )
    new_cable_speed = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["cable_length_first_derivative"]
    )
    new_trolley_motor_pwm = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["pwm_trolley_motor"]
    )
    new_hoist_motor_pwm = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["pwm_hoist_motor"]
    )
    new_trolley_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["trolley_motor_voltage"]
    )
    new_hoist_motor_voltage = np.interp(
        time_array, datasets[i]["timestamp"], datasets[i]["hoist_motor_voltage"]
    )

    interpolated_df = {
        "time": time_array,
        "trolley_position": new_trolley_position,
        "cable_length": new_cable_length,
        "sway_angle": new_sway_angle,
        "trolley_speed": new_trolley_speed,
        "cable_speed": new_cable_speed,
        "trolley_motor_pwm": new_trolley_motor_pwm,
        "hoist_motor_pwm": new_hoist_motor_pwm,
        "trolley_motor_voltage": new_trolley_motor_voltage,
        "hoist_motor_voltage": new_hoist_motor_voltage,
    }

    interpolated_datasets.append(interpolated_df)

print(len(interpolated_datasets))


39


In [6]:
max_sway_angle = 0
for i in range(len(interpolated_datasets)):
    max_sway_angle = max(
        max_sway_angle, max(abs(interpolated_datasets[i]["sway_angle"]))
    )
max_sway_angle *= 1.1

# Sort the interpolated_datasets by average trolley motor PWM from lowest to highest and then by average hoist motor PWM from lowest to highest
interpolated_datasets.sort(key=lambda x: np.mean(x["trolley_motor_pwm"]))
interpolated_datasets.sort(key=lambda x: np.mean(x["hoist_motor_pwm"]))

for key in interpolated_datasets[0]:
    print(key)

time
trolley_position
cable_length
sway_angle
trolley_speed
cable_speed
trolley_motor_pwm
hoist_motor_pwm
trolley_motor_voltage
hoist_motor_voltage


In [7]:
USE_BEST_DATASET = True
best_dataset = []
if USE_BEST_DATASET:
    for index in range(len(interpolated_datasets)):
        if (index >= 9 and index <= 20) or (index >= 30):
            continue
        best_dataset.append(interpolated_datasets[index])
    
    interpolated_datasets = best_dataset
print(len(interpolated_datasets))

18


In [8]:
SHOW_INTERPOLATED_PLOT = False
if SHOW_INTERPOLATED_PLOT:
    for index, interpolated_dataset in enumerate(interpolated_datasets):
        # if (index >= 9 and index <= 20) or (index >= 30):
        #     continue
        # Create a figure and a set of subplots
        figure, ax = plt.subplots(4, 2, figsize=(14, 9), sharex=True)
        figure.suptitle(f"Interpolated Dataset {index+1}", fontsize=16)
        ut.add_to_subplot(ax[0, 0], interpolated_datasets[index]["time"], interpolated_datasets[index]["trolley_motor_pwm"], ylabel="PWM motor troli", color="blue")
        ut.add_to_subplot(ax[0, 1], interpolated_datasets[index]["time"], interpolated_datasets[index]["hoist_motor_pwm"], ylabel="PWM motor angkut", color="red")
        ut.add_to_subplot(ax[1, 0], interpolated_datasets[index]["time"], interpolated_datasets[index]["trolley_motor_voltage"], ylabel="Tegangan motor troli (V)", color="blue")
        ut.add_to_subplot(ax[1, 1], interpolated_datasets[index]["time"], interpolated_datasets[index]["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V)", color="red")
        ut.add_to_subplot(ax[2, 0], interpolated_datasets[index]["time"], interpolated_datasets[index]["trolley_position"], ylabel="Posisi troli (m)", color="green")
        ut.add_to_subplot(ax[2, 0], interpolated_datasets[index]["time"], interpolated_datasets[index]["trolley_speed"], ylabel="Kecepatan troli (m/s)", color="green", linestyle="--")
        ut.add_to_subplot(ax[2, 1], interpolated_datasets[index]["time"], interpolated_datasets[index]["cable_length"], ylabel="Panjang tali (m)", color="orange")
        ut.add_to_subplot(ax[2, 1], interpolated_datasets[index]["time"], interpolated_datasets[index]["cable_speed"], ylabel="Kecepatan tali (m/s)", color="orange", linestyle="--")
        ut.add_to_subplot(ax[3, 0], interpolated_datasets[index]["time"], interpolated_datasets[index]["sway_angle"], "Waktu (s)", "Sudut ayun (°)", "purple")
        ut.add_to_subplot(ax[3, 1], interpolated_datasets[index]["time"], interpolated_datasets[index]["sway_angle"], "Waktu (s)", "Sudut ayun (°)", "purple")
        ax[3, 0].set_ylim(-max_sway_angle, max_sway_angle)
        ax[3, 1].set_ylim(-max_sway_angle, max_sway_angle)
        plt.tight_layout()
        plt.show()

In [9]:
# Open gantry crane parameter json file
USE_LAST_BEST_PARAMETERS = False
if USE_LAST_BEST_PARAMETERS:
    all_gantry_crane_parameters = ut.load_json(RESULTS_PATH + "best_parameters.json")
else:
    all_gantry_crane_parameters = ut.load_json(GANTRY_CRANE_PARAMETERS_JSON_PATH)["gantry_crane_system_model"]["parameters"]
print(all_gantry_crane_parameters)

{'trolley_motor_torque_constant': {'value': 0.001, 'unit': 'Nm/A', 'description': 'Torque constant of the trolley motor', 'measured': False}, 'trolley_motor_back_emf_constant': {'value': 0.001, 'unit': 'V/(rad/s)', 'description': 'Back EMF constant of the trolley motor', 'measured': False}, 'trolley_motor_rotator_inertia': {'value': 0.001, 'unit': 'kg.m^2', 'description': 'Rotational inertia of the trolley motor rotor', 'measured': False}, 'trolley_motor_damping_coefficient': {'value': 0.001, 'unit': 'Ns.m/rad', 'description': 'Damping coefficient of the trolley motor', 'measured': False}, 'trolley_damping_coefficient': {'value': 0.001, 'unit': 'Ns/m', 'description': 'Damping coefficient of the trolley', 'measured': False}, 'hoist_motor_back_emf_constant': {'value': 0.1, 'unit': 'V/(rad/s)', 'description': 'Back EMF constant of the hoist motor', 'measured': False}, 'hoist_motor_rotator_inertia': {'value': 0.001, 'unit': 'kg.m^2', 'description': 'Rotational inertia of the hoist motor ro

In [10]:
measured_parameters = {}
approximated_parameters = {}
for parameter in all_gantry_crane_parameters:
    if all_gantry_crane_parameters[parameter]["measured"]:
        measured_parameters[parameter] = all_gantry_crane_parameters[parameter]
    else:
        approximated_parameters[parameter] = all_gantry_crane_parameters[parameter]

# Print header of approximated parameters
print("Approximated Parameters")
print("Name", "Value")
for parameter in approximated_parameters:
    print(parameter, approximated_parameters[parameter]["value"])

Approximated Parameters
Name Value
trolley_motor_torque_constant 0.001
trolley_motor_back_emf_constant 0.001
trolley_motor_rotator_inertia 0.001
trolley_motor_damping_coefficient 0.001
trolley_damping_coefficient 0.001
hoist_motor_back_emf_constant 0.1
hoist_motor_rotator_inertia 0.001
hoist_motor_torque_constant 10.001
hoist_motor_damping_coefficient 0.001
cable_damping_coefficient 0.001


In [11]:
# for parameter in all_gantry_crane_parameters:
# #     # Continue if parameter have "trolley_*" or "hoist_*" in its name
#     # if "trolley_" in parameter:
#     #     continue
#     if "hoist_" in parameter or "cable_" in parameter:
#         continue
#     if parameter in approximated_parameters:
#         approximated_parameters.pop(parameter)

for parameter in approximated_parameters:
    print(parameter)

trolley_motor_torque_constant
trolley_motor_back_emf_constant
trolley_motor_rotator_inertia
trolley_motor_damping_coefficient
trolley_damping_coefficient
hoist_motor_back_emf_constant
hoist_motor_rotator_inertia
hoist_motor_torque_constant
hoist_motor_damping_coefficient
cable_damping_coefficient


In [12]:
from model import Simulator
simulator = Simulator(DT, NUM_STEPS)

In [13]:
def cost_function(parameter, datasets):
    # NUMBER_OF_DATASETS = len(datasets)
    average_sum_RMSE_squared = 0
    for dataset in datasets:
        trolley_motor_input = dataset["trolley_motor_pwm"]
        hoist_motor_input = dataset["hoist_motor_pwm"]
        initial_conditions = {
            "x" : dataset["trolley_position"][0],
            "l" : dataset["cable_length"][0],
            "theta" : dataset["sway_angle"][0],
        }
        simulator.simulate(parameter, trolley_motor_input, hoist_motor_input, initial_conditions=initial_conditions)
        simulation_result = simulator.get_results()

        simulation_result_array = [simulation_result[key] for key in simulation_result]
        dataset_array = [dataset[key] for key in dataset]

        sum_RMSE = ut.calculate_sum_root_mean_squared_errors(simulation_result_array, dataset_array)
        
        if np.isnan(sum_RMSE):
            return np.inf
        
        average_sum_RMSE_squared += sum_RMSE**2
    
    # average_sum_RMSE_squared /= NUMBER_OF_DATASETS
    
    return average_sum_RMSE_squared

In [14]:
optimize_range = {
    "trolley_mass": (0.001, 10.0),  # Done
    "trolley_damping_coefficient": (0.001, 10.0),  # Done
    "cable_damping_coefficient": (0.001, 10.0),  # Done
    "trolley_motor_rotator_inertia": (0.001, 10.0),  # Done
    "trolley_motor_damping_coefficient": (0.001, 10.0),
    "trolley_motor_back_emf_constant": (0.001, 10.0),  # Done
    "trolley_motor_torque_constant": (0.001, 10.0),  # Done
    "hoist_motor_rotator_inertia": (0.001, 10.0),  # Done
    "hoist_motor_damping_coefficient": (0.001, 10.0), # Done
    "hoist_motor_back_emf_constant": (0.001, 10.0),  # Done
    "hoist_motor_torque_constant": (0.001, 10.0),  # Done
    "trolley_motor_activation_threshold_voltage": (0.001, 10.0),  # Done
    "hoist_motor_activation_threshold_voltage": (0.001, 10.0),  # Done
}

In [15]:
MAX_EPISODES = 3
MAX_GENERATIONS = 500
NUMBER_OF_POPULATIONS = 20
SWITCH_PROBABILITY = 0.8
LAMBDA = 1.5
NUMBER_OF_PARAMETERS = len(approximated_parameters)
BATCH_SIZE = len(interpolated_datasets)

In [16]:
def Levy_flight(Lambda):
    step_size = np.zeros(NUMBER_OF_PARAMETERS)
    # Levy flight
    sigma = (math.gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) / (math.gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    for i in range(NUMBER_OF_PARAMETERS):
        u = np.random.normal(0, sigma)
        v = np.random.normal(0, 1)
        step = u / np.abs(v) ** (1 / Lambda)
        step_size[i] = step
    return step_size

In [17]:
random_dataset_indexes = random.sample(range(len(interpolated_datasets)), BATCH_SIZE)
random_datasets = [interpolated_datasets[index] for index in random_dataset_indexes]

In [18]:
best_cost_histories = []
for episode in range(MAX_EPISODES):
    # Generate new population_of_parameters
    print(f"\033[92mCreating new population for episode {episode+1}.\033[0m Population size: {NUMBER_OF_POPULATIONS}")
    population_of_parameters = []
    for i in range(NUMBER_OF_POPULATIONS):
        new_parameters = copy.deepcopy(all_gantry_crane_parameters)
        for parameter in approximated_parameters:
            new_parameters[parameter]["value"] = np.random.uniform(optimize_range[parameter][0], optimize_range[parameter][1])
        population_of_parameters.append(new_parameters)
        
    # Find best cost in population_of_parameters
    print(f"\033[92mFinding initial best cost for episode {episode+1}.\033[0m", end=" ")
    costs = np.zeros(NUMBER_OF_POPULATIONS)
    for index, parameters in enumerate(population_of_parameters):
        costs[index] = cost_function(parameters, random_datasets)
    
    best_parameters_index = np.argmin(costs)
    best_cost = costs[best_parameters_index]
    best_parameters = population_of_parameters[best_parameters_index]
    print(f"Initial best cost: {best_cost}")

    best_cost_history = np.zeros(MAX_GENERATIONS + 1)
    best_cost_history[0] = best_cost
    for generation in range(MAX_GENERATIONS):
        for index, parameters in enumerate(population_of_parameters):
            # Generate temporary parameters
            new_parameters = copy.deepcopy(parameters)
            polination_type = ""
            if np.random.uniform(0, 1) < SWITCH_PROBABILITY:
                polination_type = "global"
                # Do global pollination
                Levy_step = Levy_flight(LAMBDA)
                step_size = Levy_step
                for index_, parameter in enumerate(approximated_parameters):
                    new_parameters[parameter]["value"] += step_size[index_] * (parameters[parameter]["value"] - best_parameters[parameter]["value"])
                    
            else:
                polination_type = "local"
                # Do local pollination
                epsilon = np.random.uniform(0, 1)
                step_size = epsilon
                j, k = random.sample(range(NUMBER_OF_POPULATIONS), 2)
                for i, parameter in enumerate(approximated_parameters):
                    new_parameters[parameter]["value"] += step_size * (population_of_parameters[j][parameter]["value"] - population_of_parameters[k][parameter]["value"])
            
            # Check if new_parameters is within the range
            for parameter in approximated_parameters:
                if new_parameters[parameter]["value"] < optimize_range[parameter][0]:
                    new_parameters[parameter]["value"] = optimize_range[parameter][0]
                elif new_parameters[parameter]["value"] > optimize_range[parameter][1]:
                    new_parameters[parameter]["value"] = optimize_range[parameter][1]
            
            # Evaluate new_parameters
            new_cost = cost_function(new_parameters, random_datasets)
            if new_cost == np.inf:
                print(f"Population {index+1} has inf cost. Generating new random parameters for population {index+1}")
                for parameter in approximated_parameters:
                    new_parameters[parameter]["value"] = np.random.uniform(optimize_range[parameter][0], optimize_range[parameter][1])
                new_cost = cost_function(new_parameters, random_datasets)
            
            if new_cost < costs[index]:
                population_of_parameters[index] = new_parameters
                costs[index] = new_cost
                
            if new_cost < best_cost:
                best_cost_history[generation+1] = new_cost
                print(f"\033[92mFound new best cost: {new_cost}.\033[0m", end=" Parameters: ")
                for parameter in best_parameters:
                    print(parameter, best_parameters[parameter]["value"], end=", ")
                print()
                best_cost = new_cost
                best_parameters = new_parameters
                ut.save_json(best_parameters, RESULTS_PATH + "best_parameters.json")
            
            print(f"Episode {episode+1} of {MAX_EPISODES}, Generation {generation+1} of {MAX_GENERATIONS}, Population {index+1}, Polination type: {polination_type}, Cost: {costs[index]}")
        
        if (generation+1) % int(MAX_GENERATIONS/10) == 0:
            print(f"\033[92mCheckpoint: Episode {episode+1}, Generation {generation+1}, Best Cost: {best_cost} \033[0m")
            for parameter in best_parameters:
                print(parameter, best_parameters[parameter]["value"], end=", ")
            print()
    
    best_cost_histories.append(best_cost_history)

Creating new population for episode 1. Population size: 20
Finding initial best cost for episode 1. 

In [ ]:
for episode, best_cost_history in enumerate(best_cost_histories):
    plt.plot(best_cost_history, label=f"Episode {episode + 1}")
    plt.title(f"Cost History of Episode {episode + 1}")
    plt.xlabel("Generation")
    plt.ylabel("Cost")
    plt.legend()
    plt.show()

In [ ]:
# Load the best parameters
best_parameters = ut.load_json(RESULTS_PATH + "best_parameters.json")

# best_parameters = ut.load_json(GANTRY_CRANE_PARAMETERS_JSON_PATH)["gantry_crane_system_model"]["parameters"]

for parameter in best_parameters:
    print(parameter, best_parameters[parameter]["value"], end=", ")
print()

# Simulate the gantry crane system using the best parameters
total_cost = 0
costs = []
for index, dataset in enumerate(interpolated_datasets):
    # if index != 0:
    #     continue
    # if index != 10:
    #     continue
    simulator.simulate(best_parameters, dataset["trolley_motor_pwm"], dataset["hoist_motor_pwm"], initial_conditions={"x": dataset["trolley_position"][0], "l": dataset["cable_length"][0], "theta": dataset["sway_angle"][0]})
    simulation_result = simulator.get_results()

    cost = cost_function(best_parameters, [dataset])
    total_cost += cost
    costs.append(cost)

    print(f"Data {index+1} - Fungsi biaya: {round(cost, 5)}")

    # Create a figure and a set of subplots
    figure, ax = plt.subplots(4, 2, figsize=(12, 6), sharex=True)
    plt.rcParams.update({"font.size": 10})
    figure.suptitle(f"Data {index+1} - Fungsi biaya: {round(cost, 5)}", fontsize=12)
    ut.add_to_subplot(ax[0, 0], simulation_result["time"], simulation_result["trolley_motor_pwm"], ylabel="PWM motor troli (simulasi)", color=plt.get_cmap("tab20")(0))
    ut.add_to_subplot(ax[0, 1], simulation_result["time"], simulation_result["hoist_motor_pwm"], ylabel="PWM motor angkut (simulasi)", color=plt.get_cmap("tab20")(2))
    ut.add_to_subplot(ax[1, 0], simulation_result["time"], simulation_result["trolley_motor_voltage"], ylabel="Tegangan motor troli (V) (simulasi)", color=plt.get_cmap("tab20")(4))
    ut.add_to_subplot(ax[1, 1], simulation_result["time"], simulation_result["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V) (simulasi)", color=plt.get_cmap("tab20")(6))
    ut.add_to_subplot(ax[2, 0], simulation_result["time"], simulation_result["trolley_position"], ylabel="Posisi troli (m) (simulasi)", color=plt.get_cmap("tab20")(8))
    ut.add_to_subplot(ax[2, 0], simulation_result["time"], simulation_result["trolley_speed"], ylabel="Kecepatan troli (m/s) (simulasi)", color=plt.get_cmap("tab20")(8), linestyle="--")
    ut.add_to_subplot(ax[2, 1], simulation_result["time"], simulation_result["cable_length"], ylabel="Panjang tali (m) (simulasi)", color=plt.get_cmap("tab20")(10))
    ut.add_to_subplot(ax[2, 1], simulation_result["time"], simulation_result["cable_speed"], ylabel="Kecepatan tali (m/s) (simulasi)", color=plt.get_cmap("tab20")(10), linestyle="--")
    ut.add_to_subplot(ax[3, 0], simulation_result["time"], simulation_result["sway_angle"], "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))
    ut.add_to_subplot(ax[3, 1], simulation_result["time"], simulation_result["sway_angle"], "Waktu (s)", "Sudut ayun (°) (simulasi)", color=plt.get_cmap("tab20")(12))

    ut.add_to_subplot(ax[0, 0], dataset["time"], dataset["trolley_motor_pwm"], ylabel="PWM motor troli (data)", color=plt.get_cmap("tab20")(1))
    ut.add_to_subplot(ax[0, 1], dataset["time"], dataset["hoist_motor_pwm"], ylabel="PWM motor angkut (data)", color=plt.get_cmap("tab20")(3))
    ut.add_to_subplot(ax[1, 0], dataset["time"], dataset["trolley_motor_voltage"], ylabel="Tegangan motor troli (V) (data)", color=plt.get_cmap("tab20")(5))
    ut.add_to_subplot(ax[1, 1], dataset["time"], dataset["hoist_motor_voltage"], ylabel="Tegangan motor angkut (V) (data)", color=plt.get_cmap("tab20")(7))
    ut.add_to_subplot(ax[2, 0], dataset["time"], dataset["trolley_position"], ylabel="Posisi troli (m) (data)", color=plt.get_cmap("tab20")(9))
    ut.add_to_subplot(ax[2, 0], dataset["time"], dataset["trolley_speed"], ylabel="Kecepatan troli (m/s) (data)", color=plt.get_cmap("tab20")(9), linestyle="--")
    ut.add_to_subplot(ax[2, 1], dataset["time"], dataset["cable_length"], ylabel="Panjang tali (m) (data)", color=plt.get_cmap("tab20")(11))
    ut.add_to_subplot(ax[2, 1], dataset["time"], dataset["cable_speed"], ylabel="Kecepatan tali (m/s) (data)", color=plt.get_cmap("tab20")(11), linestyle="--")
    ut.add_to_subplot(ax[3, 0], dataset["time"], dataset["sway_angle"], "Waktu (s)", "Sudut ayun (°) (data)", color=plt.get_cmap("tab20")(13))
    ut.add_to_subplot(ax[3, 1], dataset["time"], dataset["sway_angle"], "Waktu (s)", "Sudut ayun (°) (data)", color=plt.get_cmap("tab20")(13))
    # ax[3, 0].set_ylim(-max_sway_angle, max_sway_angle)
    # ax[3, 1].set_ylim(-max_sway_angle, max_sway_angle)
    plt.tight_layout()

    # Save the figure as SVG
    plt.savefig(RESULTS_PATH + f"figure/data_{index+1}.svg")
    plt.show()

print("Minimum cost: ", np.min(costs))
print("Maximum cost: ", np.max(costs))
print(f"Total cost: {total_cost}")